In [34]:
# read it in to inspect it
with open('bhagavad_geeta_input_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [38]:
print(text[:10000])

Chapter 1 Shlok 1
Translation : धृतराष्ट्र बोले- हे संजय! धर्मभूमि कुरुक्षेत्र में एकत्रित, युद्ध की इच्छावाले मेरे और पाण्डु के पुत्रों ने क्या किया?
Commentary : 'धर्मक्षेत्रे' 'कुरुक्षेत्रे'-- कुरुक्षेत्र में देवताओं ने यज्ञ किया था। राजा कुरु ने भी यहाँ तपस्या की थी। यज्ञादि धर्ममय कार्य होने से तथा राजा कुरु की तपस्याभूमि होने से इसको धर्मभूमि कुरुक्षेत्र कहा गया है। यहाँ ॓'धर्मक्षेत्रे' और 'कुरुक्षेत्रे' पदों में 'क्षेत्र' शब्द देने में धृतराष्ट्र का अभिप्राय है कि यह अपनी कुरुवंशियों की भूमि है। यह केवल लड़ाई की भूमि ही नहीं है, प्रत्युत तीर्थभूमि भी है, जिसमें प्राणी जीते-जी पवित्र कर्म करके अपना कल्याण कर सकते हैं। इस तरह लौकिक और पारलौकिक सब तरह का लाभ हो जाय-- ऐसा विचार करके एवं श्रेष्ठ पुरुषों की सम्मति लेकर ही युद्ध के लिये यह भूमि चुनी गयी है। संसार में प्रायः तीन बातों को लेकर लड़ाई होती है-- भूमि, धन और स्त्री। इस तीनों में भी राजाओं का आपस में लड़ना मुख्यतः जमीन को लेकर होता है। यहाँ 'कुरुक्षेत्रे' पद देने का तात्पर्य भी जमीन को लेकर ल़ड़ने में है। कुरुवंश में धृतराष्ट

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3218403


In [3]:
vocab = sorted(list(set(text)))
print('Length of Vocabulary:',len(vocab))

Length of Vocabulary: 136


In [4]:
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print("length of text:", len(text))
print("length of tokens:", len(tokens))

length of text: 3218403
length of tokens: 8340297


In [5]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [6]:
stats = get_stats(tokens)

In [7]:
list(stats.items())[:5]

[((67, 104), 698),
 ((104, 97), 698),
 ((97, 112), 698),
 ((112, 116), 698),
 ((116, 101), 698)]

In [15]:
top_pair = max(stats, key=stats.get)
top_pair

(224, 164)

In [16]:
chr(224), chr(164)

('à', '¤')

In [17]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 1000 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  # print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 8340297
ids length: 1267199
compression ratio: 6.58X


In [21]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([367, 632, 264, 263, 183, 742, 283]))

धृतराष्ट्र


In [20]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode("धृतराष्ट्र"))

[367, 632, 264, 263, 183, 742, 283]


In [22]:
text2 = decode(encode(text))
print(text2 == text)

True


In [32]:
def tokenize_text(input_text):
    tokens = encode(input_text)
    return tokens

In [33]:
sample_text_input = "'धर्मक्षेत्रे' 'कुरुक्षेत्रे'-- कुरुक्षेत्र में देवताओं ने यज्ञ किया था। राजा कुरु ने भी यहाँ तपस्या की थी। यज्ञादि धर्ममय कार्य होने से तथा राजा कुरु की तपस्याभूमि होने से इसको धर्मभूमि कुरुक्षेत्र कहा गया है। यहाँ ॓'धर्मक्षेत्रे' और 'कुरुक्षेत्रे' पदों में 'क्षेत्र' शब्द देने में धृतराष्ट्र का अभिप्राय है कि यह अपनी कुरुवंशियों की भूमि है। यह केवल लड़ाई की भूमि ही नहीं है, प्रत्युत तीर्थभूमि भी है, जिसमें प्राणी जीते-जी पवित्र कर्म करके अपना कल्याण कर सकते हैं। इस तरह लौकिक और पारलौकिक सब तरह का लाभ हो जाय-- ऐसा विचार करके एवं श्रेष्ठ पुरुषों की सम्मति लेकर ही युद्ध के लिये यह भूमि चुनी गयी है। संसार में प्रायः तीन बातों को लेकर लड़ाई होती है-- भूमि, धन और स्त्री। इस तीनों में भी राजाओं का आपस में लड़ना मुख्यतः जमीन को लेकर होता है। यहाँ 'कुरुक्षेत्रे' पद देने का तात्पर्य भी जमीन को लेकर ल़ड़ने में है। कुरुवंश में धृतराष्ट्र और पाण्डु के पुत्र सब एक हो जाते हैं। कुरुवंशी होने से दोनों का कुरुक्षेत्र में अर्थात् राजा कुरु की जमीन पर समान हक लगता है। इसलिये (कौरवों द्वारा पाण्डवों को उनकी जमीन न देने के कारण) दोनों जमीन के लिये लड़ाई करने आये हुए हैं। यद्यपि अपनी भूमि होने के कारण दोनों के लिये 'कुरुक्षेत्रे' पद देना युक्तिसंगत, न्यायसंगत है, तथापि हमारी सनातन वैदिक संस्कृति ऐसी विलक्षण है कि कोई भी कार्य करना होता है, तो वह धर्म को सामने रखकर ही होता है। युद्ध-जैसा कार्य भी धर्मभूमि-- तीर्थभूमि में ही करते हैं, जिससे युद्ध में मरने वालों का उद्धार हो जाय, कल्याण हो जाय। अतः यहाँ कुरुक्षेत्र के साथ 'धर्मक्षेत्रे' पद आया है। यहाँ आरम्भ में 'धर्म' पद से एक और बात भी मालूम होती है। अगर आरम्भ के 'धर्म' पद में से 'धर्' लिया जाय और अठारहवें अध्याय के अन्तिम श्लोक के 'मम' पदों से 'म' लिया जाय, तो 'धर्म' शब्द बन जाता है। अतः सम्पूर्ण गीता धर्म के अन्तर्गत है अर्थात् धर्म का पालन करने से गीता के सिद्धान्तों का पालन हो जाता है और गीता के सिद्धान्तों के अनुसार कर्तव्य कर्म करने से धर्म का अनुष्ठान हो जाता है। इन 'धर्मक्षेत्रे कुरुक्षेत्रे' पदों से सभी मनुष्यों को यह शिक्षा लेनी चाहिये कि कोई भी काम करना हो तो वह धर्म को सामने रखकर ही करना चाहिये। प्रत्येक कार्य सबके हित की दृष्टि से ही करना चाहिये, केवल अपने सुख-आराम-की दृष्टि से नहीं; और कर्तव्य-अकर्तव्य के विषय में शास्त्र को सामने रखना चाहिये (गीता 16। 24)। 'समवेता युयुत्सवः'-- राजाओं के द्वारा बारबार सन्धि का प्रस्ताव रखने पर भी दुर्योधन ने सन्धि करना स्वीकार नहीं किया। इतना ही नहीं, भगवान् श्रीकृष्ण के कहने पर भी मेरे पुत्र दुर्योधन ने स्पष्ट कह दिया कि बिना युद्ध के मैं तीखी सूई की नोक-जितनी जमीन भी पाण्डवों को नहीं दूँगा। (टिप्पणी प0 2.1) तब मजबूर होकर पाण्डवों ने भी युद्ध करना स्वीकार किया है। इस प्रकार मेरे पुत्र और पाण्डुपुत्र-- दोनों ही सेनाओं के सहित युद्ध की इच्छा से इकट्ठे हुए हैं। दोनों सेनाओं में युद्ध की इच्छा रहने पर भी दुर्योधन में युद्ध की इच्छा विशेषरूप से थी। उसका मुख्य उद्देश्य राज्य-प्राप्ति का ही था। वह राज्य-प्राप्ति धर्म से हो चाहे अधर्म से, न्याय से हो चाहे अन्याय से, विहित रीति से हो चाहे निषिद्ध रीति से, किसी भी तरह से हमें राज्य मिलना चाहिये-- ऐसा उसका भाव था। इसलिये विशेषरूप से दुर्योधन का पक्ष ही युयुत्सु अर्थात् युद्ध की इच्छावाला था। पाण्डवों में धर्म की मुख्यता थी। उनका ऐसा भाव था कि हम चाहे जैसा जीवन-निर्वाह कर लेंगे, पर अपने धर्म में बाधा नहीं आने देंगे, धर्म के विरुद्ध नहीं चलेंगे। इस बात को लेकर महाराज युधिष्ठिर युद्ध नहीं करना चाहते थे। परन्तु जिस माँ की आज्ञा से युधिष्ठिर ने चारों भाइयों सहित द्रौपदी से विवाह किया था, उस माँ की आज्ञा होने के कारण ही महाराज युधिष्ठिर की युद्ध में प्रवृत्ति हुई थी (टिप्पणी प0 2.2) अर्थात् केवल माँ के आज्ञा-पालनरूप धर्म से ही युधिष्ठिर युद्ध की इच्छावाले हुये हैं। तात्पर्य है कि दुर्योधन आदि तो राज्य को लेकर ही युयुत्सु थे, पर पाण्डव धर्म को लेकर ही युयुत्सु बने थे। 'मामकाः पाण्डवाश्चैव'-- पाण्डव धृतराष्ट्र को (अपने पिता के बड़े भाई होने से) पिता के समान समझते थे और उनकी आज्ञा का पालन करते थे। धृतराष्ट्र के द्वारा अनुचित आज्ञा देने पर भी पाण्डव उचित-अनुचित का विचार न करके उनकी आज्ञा का पालन करते थे। अतः यहाँ 'मामकाः' पद के अन्तर्गत कौरव (टिप्पणी प0 3.1) और पाण्डव दोनों आ जाते हैं। फिर भी 'पाण्डवाः' पद अलग देने का तात्पर्य है कि धृतराष्ट्र का अपने पुत्रों में तथा पाण्डुपुत्रों में समान भाव नहीं था। उनमें पक्षपात था,अपने पुत्रों के प्रति मोह था। वे दुर्योधन आदि को तो अपना मानते थे, पर पाण्डवों को अपना नहीं मानते थे। (टिप्पणी प0 3.2) इस कारण उन्होंने अपने पुत्रों के लिये 'मामकाः' और पाण्डुपुत्रों के लिये 'पाण्डवा' पद का प्रयोग किया है; क्योंकि जो भाव भीतर होते हैं, वे ही प्रायः वाणी से बाहर निकलते हैं। इस द्वैधीभाव के कारण ही धृतराष्ट्र को अपने कुल के संहार का दुःख भोगना पड़ा। इससे मनुष्यमात्र को यह शिक्षा लेनी चाहिये कि वह अपने घरों में, मुहल्लों में, गाँवों में, प्रान्तों में, देशों में, सम्प्रदायों में द्वैधीभाव अर्थात् ये अपने हैं, ये दूसरे हैं-- ऐसा भाव न रखे। कारण कि द्वैधीभाव से आपस में प्रेम, स्नेह नहीं होता, प्रत्युत कलह होती है। यहाँ 'पाण्डवाः' पद के साथ 'एव' पद देने का तात्पर्य है कि पाण्डव तो बड़े धर्मात्मा हैं; अतः उन्हें युद्ध नहीं करना चाहिये था। परन्तु वे भी युद्ध के लिये रणभूमि में आ गये तो वहाँ आकर उन्होंने क्या किया? 'मामकाः' और 'पाण्डवाः' (टिप्पणी प0 3.3) इनमें से पहले 'मामकाः' पद का उत्तर सञ्जय आगे के (दूसरे) श्लोक से तेरहवें श्लोक तक देंगे कि आपके पुत्र दुर्योधन ने पाण्डवों की सेना को देखकर द्रोणाचार्य के मन में पाण्डवों के प्रति द्वेष पैदा करने के लिये उनके पास जाकर पाण्डवों के मुख्य-मुख्य सेनापतियों के नाम लिये। उसके बाद दुर्योधन ने अपनी सेना के मुख्य-मुख्य योद्धाओं के नाम लेकर उनके रण-कौशल आदि की प्रशंसा की। दुर्योधन को प्रसन्न करने के लिये भीष्मजी ने जोर से शंख बजाया। उसको सुनकर कौरव-सेना में शंख आदि बाजे बज उठे। फिर चौदहवें श्लोक से उन्नीसवें श्लोक तक 'पाण्डवाः' पद का उत्तर देंगे कि रथ में बैठे हुए पाण्डवपक्षीय श्रीकृष्ण ने शंख बजाया। उसके बाद अर्जुन, भीम, युधिष्ठिर, नकुल, सहदेव आदि ने अपने-अपने शंख बजाये, जिससे दुर्योधन की सेना का हृदय दहल गया। उसके बाद भी सञ्जय पाण्डवों की बात कहते-कहते बीसवें श्लोक से श्रीकृष्ण और अर्जुन के संवाद का प्रसङ्ग आरम्भ कर देंगे। 'किमकुर्वत'-- 'किम्' शब्द के तीन अर्थ होते हैं-- विकल्प, निन्दा (आक्षेप) और प्रश्न। युद्ध हुआ कि नहीं? इस तरह का विकल्प तो यहाँ लिया नहीं जा सकता; क्योंकि दस दिन तक युद्ध हो चुका है और भीष्म जी को रथ से गिरा देने के बाद सञ्जय हस्तिनापुर आकर धृतराष्ट्र को वहाँ की घटना सुना रहे हैं। 'मेरे और पाण्डु के पुत्रों ने यह क्या किया, जो कि युद्ध कर बैठे! उनको युद्ध नहीं करना चाहिये था'-- ऐसी निन्दा या आक्षेप भी यहाँ नहीं लिया जा सकता; क्योंकि युद्ध तो चल ही रहा था और धृतराष्ट्र के भीतर भी आक्षेपपूर्वक पूछने का भाव नहीं था। यहाँ 'किम्' शब्द का अर्थ प्रश्न लेना ही ठीक बैठता है। धृतराष्ट्र सञ्जय से भिन्न-भिन्न प्रकार की छोटी-बड़ी सब घटनाओं को अनुक्रम से विस्तारपूर्वक ठीक-ठीक जानने के लिये ही प्रश्न कर रहे हैं। सम्बन्ध-- धृतराष्ट्र के प्रश्न का उत्तर सञ्जय आगे के श्लोक से देना आरम्भ करते हैं।"

print(tokenize_text(sample_text_input))
print('--------------------------------')
print(len(tokenize_text(sample_text_input)))

[39, 367, 364, 414, 906, 262, 39, 592, 267, 405, 279, 414, 906, 262, 794, 268, 405, 279, 414, 906, 434, 446, 289, 270, 263, 147, 290, 168, 274, 175, 636, 555, 277, 827, 320, 176, 572, 607, 405, 279, 318, 274, 461, 773, 336, 291, 276, 783, 430, 165, 273, 320, 175, 636, 475, 265, 633, 364, 280, 305, 268, 710, 934, 184, 274, 929, 176, 572, 607, 405, 279, 430, 164, 291, 276, 284, 979, 303, 280, 265, 934, 184, 274, 339, 454, 167, 364, 348, 303, 280, 265, 268, 405, 279, 414, 906, 429, 599, 738, 773, 32, 257, 147, 39, 367, 364, 414, 906, 262, 39, 487, 592, 267, 405, 279, 414, 906, 262, 39, 587, 490, 724, 592, 414, 906, 39, 946, 531, 361, 588, 167, 632, 264, 263, 183, 742, 283, 355, 964, 282, 409, 363, 546, 458, 420, 780, 268, 405, 279, 289, 272, 337, 568, 430, 173, 303, 280, 265, 622, 441, 677, 459, 469, 263, 136, 430, 173, 303, 280, 265, 323, 389, 892, 805, 875, 336, 273, 427, 348, 303, 280, 265, 687, 892, 575, 395, 820, 293, 156, 273, 344, 45, 349, 868, 289, 915, 432, 869, 445, 462, 268, 31

In [28]:
import pickle

with open('merges.pkl', 'wb') as file:
    pickle.dump(merges, file)
with open('vocab.pkl', 'wb') as file:
    pickle.dump(vocab, file)